## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

#get config variable from folder above
os.chdir("..")
from config import g_key
os.chdir("Vacation_Search")

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the Weather_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sorland,NO,67.6670,12.6934,37.44,56,100,22.46,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,68.25,96,100,5.75,moderate rain
2,2,Wattegama,LK,6.7989,81.4808,76.93,91,99,1.86,overcast clouds
3,3,Eureka,US,40.8021,-124.1637,51.21,72,100,10.00,overcast clouds
4,4,Tambovka,RU,50.0969,128.0489,26.62,92,58,1.97,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp_in = float(input("What is the minimum temperature you would like for your trip? "))
max_temp_in = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_raw_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp_in) & \
                                       (city_data_df["Max Temp"] >= min_temp_in)]
preferred_cities_raw_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,68.25,96,100,5.75,moderate rain
2,2,Wattegama,LK,6.7989,81.4808,76.93,91,99,1.86,overcast clouds
6,6,Quatre Cocos,MU,-20.2078,57.7625,78.10,84,59,15.64,light rain
10,10,Mar Del Plata,AR,-38.0023,-57.5575,66.22,95,75,11.01,light rain
20,20,Shu,KZ,43.5983,73.7614,62.02,53,77,12.19,broken clouds
21,21,Chuy,UY,-33.6971,-53.4616,76.28,54,1,10.20,clear sky
24,24,Tsabong,BW,-26.0217,22.4010,75.61,39,80,1.57,broken clouds
25,25,Rikitea,PF,-23.1203,-134.9692,79.27,73,0,17.40,clear sky
26,26,Hays,US,30.0502,-98.0336,70.30,19,0,4.00,clear sky
31,31,Mareeba,AU,-17.0000,145.4333,74.12,92,9,2.51,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_raw_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_raw_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_raw_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_raw_df["Hotel Name"] = ""
hotel_raw_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,68.25,moderate rain,19.7297,-155.0900,
2,Wattegama,LK,76.93,overcast clouds,6.7989,81.4808,
6,Quatre Cocos,MU,78.10,light rain,-20.2078,57.7625,
10,Mar Del Plata,AR,66.22,light rain,-38.0023,-57.5575,
20,Shu,KZ,62.02,broken clouds,43.5983,73.7614,
21,Chuy,UY,76.28,clear sky,-33.6971,-53.4616,
24,Tsabong,BW,75.61,broken clouds,-26.0217,22.4010,
25,Rikitea,PF,79.27,clear sky,-23.1203,-134.9692,
26,Hays,US,70.30,clear sky,30.0502,-98.0336,
31,Mareeba,AU,74.12,clear sky,-17.0000,145.4333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_raw_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_raw_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_raw_df.count())
hotel_df=hotel_raw_df.loc[hotel_raw_df["Hotel Name"] != ""]
print(hotel_df.count())
hotel_df.head()

City                   266
Country                266
Max Temp               266
Current Description    266
Lat                    266
Lng                    266
Hotel Name             266
dtype: int64
City                   251
Country                251
Max Temp               251
Current Description    251
Lat                    251
Lng                    251
Hotel Name             251
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,68.25,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Quatre Cocos,MU,78.10,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
10,Mar Del Plata,AR,66.22,light rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
20,Shu,KZ,62.02,broken clouds,43.5983,73.7614,Shyngyzkhan
21,Chuy,UY,76.28,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza


In [15]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("Vacation_Search","WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))